In [1]:
import pandas as pd
import os
import gzip
import numpy as np
import json
import ast
import geopandas
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, MultiPolygon
from haversine import haversine, Unit
from shapely import wkt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity

year = 2018

In [2]:
table = pd.read_csv(r'F:\safegraph\presentation_1105\table_comprehensive.csv').drop(columns = 'Unnamed: 0')

In [3]:
census_data = pd.read_csv(r'F:\safegraph\census data\census_data_by_cbg.csv').drop(columns  = 'Unnamed: 0')
census_data['cbg_geometry'] = census_data['cbg_geometry'].apply(wkt.loads)
census_gdf = geopandas.GeoDataFrame(census_data, geometry = 'cbg_geometry')

In [4]:
census_similarity = pd.read_csv(r'F:\safegraph\presentation_1105\NY_cbg_census.csv').drop(columns = ['Unnamed: 0',
                                                                                                                            'NAME'])

In [5]:
cbgs = census_data.census_block_group.tolist()

In [6]:
census_similarity = census_similarity.rename(columns = {'GEOID': 'cbg'})
census_similarity = census_similarity[census_similarity['cbg'].isin(cbgs)]

In [7]:
census_similarity = census_similarity.fillna(0)

In [8]:
def get_pro(part):
    total_list = census_similarity.populationE.tolist()
    part_list = census_similarity[part].tolist()
    ans = []
    for i,num in enumerate(total_list):
        if num == 0:
            ans.append(0)
        else:
            ans.append(part_list[i]/num)
    return ans

In [9]:
change = ['bachelor_degree_25E', 'whiteE', 'blackE', 'asianE', 'hispanicE']
for i in change:
    census_similarity[i] = get_pro(i)

In [10]:
census_similarity

,cbg,populationE,med_ageE,bachelor_degree_25E,med_hh_incomeE,whiteE,blackE,asianE,hispanicE
0,360050245023,1886,28.9,0.042418,30500.0,0.154295,0.239130,0.000000,0.873807
1,360050247002,1318,35.4,0.080425,44524.0,0.045524,0.523520,0.004552,0.502276
2,360050253004,1222,26.6,0.044190,36250.0,0.094108,0.036825,0.000000,0.945172
3,360050263005,1941,63.8,0.156620,12351.0,0.311180,0.220505,0.000000,0.459042
4,360050265004,1191,28.2,0.073048,38099.0,0.028547,0.223342,0.073887,0.748950
...,...,...,...,...,...,...,...,...,...
14060,360850207000,0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
14061,360850208013,2110,51.9,0.176777,54978.0,0.933175,0.000000,0.066825,0.045498
14062,360850003000,0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
14063,360850006002,1995,55.4,0.306767,51964.0,0.696241,0.117293,0.112281,0.198997


In [11]:
census_similarity_compute = census_similarity.drop(columns = ['cbg', 'populationE'])

In [12]:
similarities = pd.DataFrame(cosine_similarity(census_similarity_compute, census_similarity_compute))

In [13]:
cbgs_in_queue = census_similarity.cbg.tolist()
def get_cos_sim(cbg_1, cbg_2):
    index_1 = cbgs_in_queue.index(cbg_1)
    index_2 = cbgs_in_queue.index(cbg_2)
    return similarities.iloc[index_1, index_2]

In [14]:
with open('F:\\safegraph\\nyc_cbgs.json') as f:
    data = json.load(f)
nyc_list = data['features']
nyc_cbg = []
for i in nyc_list:
    nyc_cbg.append(i['properties']['GEOID'])
    
def get_stores_foot(group, y):
    if group == 'Target':
        walk_path_filtered = r'F:\safegraph\matrix_0920_Target'
    else:
        walk_path_filtered = r'F:\safegraph\competitors_1030\matrix'
    datapath_all_filtered = os.listdir(walk_path_filtered)
    datapath_all_filtered = [walk_path_filtered +'\\'+i for i in datapath_all_filtered]
    todo_list = [i for i in datapath_all_filtered if str(y) in i]
    if group == 'Target':
        data_filtered = [pd.read_csv(i).rename(columns = {'Unnamed: 0.1': 'cbg'}).set_index('cbg').drop(columns = ['Unnamed: 0']) 
                         for i in todo_list]
    else:
        data_filtered = [pd.read_csv(i).rename(columns = {'Unnamed: 0': 'cbg'}). set_index('cbg') for i in todo_list]
    final_data = sum(data_filtered).reset_index()
    return final_data

target_visitors = get_stores_foot('Target', year)
competitors_visitors = get_stores_foot('c', year)
visitors = pd.merge(target_visitors, competitors_visitors, on='cbg')

In [15]:
shp_path = 'F:\\safegraph\\tl_2017_36_bg\\tl_2017_36_bg.shp'
shp_df = geopandas.GeoDataFrame.from_file(shp_path,encoding = 'gb18030')
cbg_info = {}
cbg_id = shp_df['GEOID'].tolist()
cbg_geo = shp_df['geometry'].tolist()
for i in range(len(cbg_id)):
    if int(cbg_id[i]) in cbgs:
        cbg_info[cbg_id[i]] = cbg_geo[i]

In [16]:
poi_nyc = pd.read_csv('F:\safegraph\sample_data\core_USA\poi_NYC.csv')
square_data = pd.read_csv(r'F:\safegraph\sample_data\SafeGraphPlacesGeoSupplementSquareFeet.csv')
stores_with_poi_data = poi_nyc[poi_nyc['safegraph_place_id'].isin(visitors.columns.tolist())][['safegraph_place_id', 'longitude', 'latitude']]
square_data = square_data[square_data['safegraph_place_id'].isin(visitors.columns.tolist())][['safegraph_place_id', 'area_square_feet']]
geo_area_stores = pd.merge(stores_with_poi_data, square_data, on = 'safegraph_place_id')
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_517E','longitude': -73.93100799999999, 'latitude': 40.795408,
                                          'area_square_feet':169356}, ignore_index=True)
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_4024_College','longitude': -73.834744, 'latitude': 40.757749,
                                          'area_square_feet':214735}, ignore_index=True)
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_40W','longitude': -73.907558, 'latitude': 40.872776, 
                                          'area_square_feet':183593}, ignore_index=True)
def get_cbg(p1):
    for k,v in cbg_info.items():
        if p1.within(v):
            return k
    return 'No'
geometry_poi = [Point(xy) for xy in zip(geo_area_stores.longitude, geo_area_stores.latitude)]
geo_area_stores_gdf = GeoDataFrame(geo_area_stores, crs="EPSG:4326", geometry=geometry_poi)
geo_area_stores_gdf['geo'] = geo_area_stores_gdf['geometry'].apply(lambda x: (x.y, x.x))
geo_area_stores_gdf['cbg'] = geo_area_stores['geometry'].apply(lambda x: get_cbg(x))
geo_area_stores_gdf = geo_area_stores_gdf[geo_area_stores_gdf['cbg'] != 'No']
geo_area_stores_gdf['cbg'] = geo_area_stores_gdf['cbg'].astype('int64')

In [17]:
store_cbg_dict = {}
for i, cont in geo_area_stores_gdf.iterrows():
    store_cbg_dict[cont.safegraph_place_id] = cont.cbg

In [18]:
table['Demographic_similarity'] = table.apply(lambda row: get_cos_sim(row.cbg, store_cbg_dict[row.safegraph_place_id]), axis=1)

In [19]:
table.to_csv(r'F:\safegraph\presentation_1105\table_comprehensive_updated.csv')